In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210605.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice]
0,01/06/2021 19:14:17,Sexy Beast (2000),Vanilla Sky (2001),NaN,NaN,NaN,LOSE YOURSELF TO DANCE,1960s,ANIME,TAKESHI KITANO,"PRIDE MONTH, BITCH",MAKING AN ASS OUT OF YOU AND ME,[Default to Nicolas Cage]
1,01/06/2021 19:16:54,Sexy Beast (2000),Swimfan (2002),Vanilla Sky (2001),Rollerball (2002),The Life Aquatic with Steve Zissou (2004),TAKESHI KITANO,MAKING AN ASS OUT OF YOU AND ME,ANIME,1960s,LOSE YOURSELF TO DANCE,"PRIDE MONTH, BITCH",[Default to Nicolas Cage]
2,01/06/2021 19:17:46,Sexy Beast (2000),Rollerball (2002),The Life Aquatic with Steve Zissou (2004),Swimfan (2002),Vanilla Sky (2001),ANIME,TAKESHI KITANO,[Default to Nicolas Cage],MAKING AN ASS OUT OF YOU AND ME,LOSE YOURSELF TO DANCE,1960s,"PRIDE MONTH, BITCH"
3,01/06/2021 19:30:11,Vanilla Sky (2001),Swimfan (2002),Sexy Beast (2000),Rollerball (2002),The Life Aquatic with Steve Zissou (2004),"PRIDE MONTH, BITCH",MAKING AN ASS OUT OF YOU AND ME,[Default to Nicolas Cage],1960s,ANIME,TAKESHI KITANO,LOSE YOURSELF TO DANCE
4,02/06/2021 19:01:07,Sexy Beast (2000),The Life Aquatic with Steve Zissou (2004),Swimfan (2002),Rollerball (2002),Vanilla Sky (2001),ANIME,MAKING AN ASS OUT OF YOU AND ME,LOSE YOURSELF TO DANCE,1960s,TAKESHI KITANO,"PRIDE MONTH, BITCH",[Default to Nicolas Cage]
5,03/06/2021 08:19:16,The Life Aquatic with Steve Zissou (2004),Swimfan (2002),Rollerball (2002),Vanilla Sky (2001),NaN,1960s,TAKESHI KITANO,MAKING AN ASS OUT OF YOU AND ME,[Default to Nicolas Cage],NaN,NaN,NaN
6,03/06/2021 08:27:31,Swimfan (2002),Sexy Beast (2000),Vanilla Sky (2001),The Life Aquatic with Steve Zissou (2004),Rollerball (2002),ANIME,"PRIDE MONTH, BITCH",LOSE YOURSELF TO DANCE,1960s,NaN,NaN,NaN


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Sexy Beast (2000)', 'Swimfan (2002)', 'Vanilla Sky (2001)',
       'Rollerball (2002)', 'The Life Aquatic with Steve Zissou (2004)'],
      dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Sexy Beast (2000)': <Candidate('Sexy Beast (2000)')>,
 'Swimfan (2002)': <Candidate('Swimfan (2002)')>,
 'Vanilla Sky (2001)': <Candidate('Vanilla Sky (2001)')>,
 'Rollerball (2002)': <Candidate('Rollerball (2002)')>,
 'The Life Aquatic with Steve Zissou (2004)': <Candidate('The Life Aquatic with Steve Zissou (2004)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Sexy Beast (2000), Vanilla Sky (2001))>,
 <Ballot(Sexy Beast (2000), Swimfan (2002), Vanilla Sky (2001), Rollerball (2002), The Life Aquatic with Steve Zissou (2004))>,
 <Ballot(Sexy Beast (2000), Rollerball (2002), The Life Aquatic with Steve Zissou (2004), Swimfan (2002), Vanilla Sky (2001))>,
 <Ballot(Vanilla Sky (2001), Swimfan (2002), Sexy Beast (2000), Rollerball (2002), The Life Aquatic with Steve Zissou (2004))>,
 <Ballot(Sexy Beast (2000), The Life Aquatic with Steve Zissou (2004), Swimfan (2002), Rollerball (2002), Vanilla Sky (2001))>,
 <Ballot(The Life Aquatic with Steve Zissou (2004), Swimfan (2002), Rollerball (2002), Vanilla Sky (2001))>,
 <Ballot(Swimfan (2002), Sexy Beast (2000), Vanilla Sky (2001), The Life Aquatic with Steve Zissou (2004), Rollerball (2002))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                                    Votes  Status
-----------------------------------------  -------  --------
Sexy Beast (2000)                                4  Hopeful
Vanilla Sky (2001)                               1  Hopeful
Swimfan (2002)                                   1  Rejected
The Life Aquatic with Steve Zissou (2004)        1  Rejected
Rollerball (2002)                                0  Rejected

FINAL RESULT
Candidate                                    Votes  Status
-----------------------------------------  -------  --------
Sexy Beast (2000)                                5  Elected
Vanilla Sky (2001)                               2  Rejected
Swimfan (2002)                                   0  Rejected
The Life Aquatic with Steve Zissou (2004)        0  Rejected
Rollerball (2002)                                0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                          Votes  Status
-------------------------------  -------  --------
ANIME                                  3  Hopeful
TAKESHI KITANO                         1  Hopeful
1960s                                  1  Hopeful
PRIDE MONTH, BITCH                     1  Hopeful
LOSE YOURSELF TO DANCE                 1  Rejected
MAKING AN ASS OUT OF YOU AND ME        0  Rejected
[Default to Nicolas Cage]              0  Rejected

ROUND 2
Candidate                          Votes  Status
-------------------------------  -------  --------
ANIME                                  3  Hopeful
1960s                                  2  Hopeful
TAKESHI KITANO                         1  Rejected
PRIDE MONTH, BITCH                     1  Rejected
LOSE YOURSELF TO DANCE                 0  Rejected
MAKING AN ASS OUT OF YOU AND ME        0  Rejected
[Default to Nicolas Cage]              0  Rejected

FINAL RESULT
Candidate                          Votes  Status
------------